In [ ]:
import glob 
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim
from music21 import converter, instrument, note, chord


In [ ]:
def get_notes():
    
    notes = []
    
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        #print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
        
    return notes


In [23]:
notes = get_notes() 

pitchnames = sorted(set(item for item in notes))

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

vocab_size = len(note_to_int)

seq_len = 100

#trainingset = [(notes[i:i + seq_len], notes[i + seq_len])
        #for i in range(len(notes) - seq_len)]
    
in_seq = []
tar_seq = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - seq_len, 1):
    sequence_in = notes[i:i + seq_len]
    sequence_out = notes[i + seq_len]
    
    for j in range(0, seq_len, 1):
        in_seq.append(note_to_int[sequence_in[j]])
        
    tar_seq.append(note_to_int[sequence_out])
print(tar_seq)

[297, 111, 318, 337, 318, 213, 355, 311, 331, 331, 337, 318, 116, 311, 297, 355, 330, 354, 285, 298, 336, 319, 354, 338, 284, 111, 297, 318, 338, 20, 356, 319, 312, 298, 184, 343, 285, 312, 298, 317, 116, 285, 23, 356, 338, 319, 312, 298, 353, 317, 354, 17, 79, 353, 317, 354, 298, 312, 319, 337, 355, 353, 317, 354, 259, 220, 353, 317, 354, 353, 317, 356, 354, 338, 319, 332, 312, 353, 319, 317, 298, 354, 356, 338, 319, 312, 298, 353, 317, 354, 312, 319, 337, 355, 353, 317, 354, 356, 338, 319, 312, 298, 353, 317, 356, 354, 338, 319, 332, 312, 353, 319, 317, 298, 354, 298, 312, 319, 337, 355, 353, 317, 354, 355, 285, 312, 298, 353, 317, 354, 331, 331, 337, 355, 354, 330, 285, 298, 336, 319, 354, 284, 337, 297, 111, 318, 337, 318, 213, 355, 311, 331, 331, 337, 318, 116, 311, 297, 355, 330, 354, 285, 298, 336, 319, 354, 338, 284, 111, 297, 318, 338, 20, 356, 319, 312, 298, 184, 343, 285, 312, 298, 317, 116, 285, 23, 356, 338, 319, 312, 298, 220, 352, 286, 287, 357, 339, 320, 306, 286, 356, 

In [11]:
class LSTM(nn.Module):
    
    def __init__(self, input_size, embedding_dim, batch_size, hidden_dim, output_size):
        super(LSTM, self).__init__()
        
        self.batch_size = batch_size
        
        self.hidden_dim = hidden_dim
        
        self.embeddings = nn.Embedding(input_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, 2) ########
        
        self.linear = nn.Linear(hidden_dim, output_size)
        

    def init_hidden(self):

        return (torch.zeros(2, self.batch_size, self.hidden_dim),
                torch.zeros(2, self.batch_size, self.hidden_dim))
    

    def forward(self, inputs):
        
        hidden = self.init_hidden()
        
        embeds = self.embeddings(inputs)
        
        lstm_out, hidden = lstm(
            embeds.view(input_size,1 ,-1), hidden)
        
        prediction = self.linear(lstm_out.view(input_size, -1))
        pre_scores = F.log_softmax(prediction, dim=1)
        return pre_scores 

In [16]:
input_size = seq_len

embedding_dim = 60
batch_size = 256
hidden_dim = 128
learning_rate = 0.01

model = LSTM(input_size, embedding_dim, batch_size, hidden_dim, vocab_size)

model.zero_grad()
model.hidden = model.init_hidden()

lossfunction = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)


Epochs = 1000

save_every = 10
print_every = 10

total_Loss = []

In [25]:

for e in range(Epochs+1):
    
    total_Loss.append(0)
    
    while len(in_seq) != 0:
        
        if len(in_seq) >= 100:
            
            optimizer.zero_grad()
            
            net_in = in_seq[:99]
            print(net_in)
            del in_seq[:99]

            output_seq = model(net_in)  
            target = tar_seq.pop(0)
            Loss = lossfunction(output_seq[-1], target)
            
            Loss.backward()
            optimizer.step()

            total_Loss += loss.item() 
        else:
            break
            
    if e % save_every == 0:
        torch.save(model.state_dict(), f'./net_{e}.pth')
    if e % print_every == 0:   
        print(f'Epoch {e}, total loss: {total_Loss[-1]}')

[356, 338, 319, 312, 298, 353, 317, 354, 17, 79, 353, 317, 354, 298, 312, 319, 337, 355, 353, 317, 354, 259, 220, 353, 317, 354, 353, 317, 356, 354, 338, 319, 332, 312, 353, 319, 317, 298, 354, 356, 338, 319, 312, 298, 353, 317, 354, 312, 319, 337, 355, 353, 317, 354, 356, 338, 319, 312, 298, 353, 317, 356, 354, 338, 319, 332, 312, 353, 319, 317, 298, 354, 298, 312, 319, 337, 355, 353, 317, 354, 355, 285, 312, 298, 353, 317, 354, 331, 331, 337, 355, 354, 330, 285, 298, 336, 319, 354, 284]


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)